<a href="https://colab.research.google.com/github/hakim733/AI-project/blob/main/Neural_network_hypermarametres_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf

In [9]:
!pip install keras_tuner

In [10]:
(X_train, y_train),(X_test,y_test ) = tf.keras.datasets.mnist.load_data()
(X_train.shape, y_train.shape),(X_test.shape, y_test.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

Preprocessing

In [11]:
X_train = X_train/255.0
X_test =X_test/255.0
X_train.min(), X_train.max()

(np.float64(0.0), np.float64(1.0))

Define the model


In [12]:
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28,28)))

    hp_activation = hp.Choice('activation', values=['relu', 'sigmoid'])
    hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=10, step=10000)
    hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=10, step=10000)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
    model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    return model



In [13]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='Neural network hypermarametres tuning')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
stop_early= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
tuner.search(X_train, y_train, epochs=5, validation_split=0.2, callbacks=[stop_early])

Trial 6 Complete [00h 00m 11s]
val_accuracy: 0.10599999874830246

Best val_accuracy So Far: 0.2097499966621399
Total elapsed time: 00h 01m 19s


In [16]:
best_hps =tuner.get_best_hyperparameters(num_trials=1)[0]


In [18]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=5, validation_split=0.2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1724 - loss: 2.0914 - val_accuracy: 0.1972 - val_loss: 1.7970
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.2043 - loss: 1.8045 - val_accuracy: 0.2108 - val_loss: 1.7824
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.2106 - loss: 1.7923 - val_accuracy: 0.2112 - val_loss: 1.7822
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.2081 - loss: 1.7874 - val_accuracy: 0.2095 - val_loss: 1.7812
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.2082 - loss: 1.7840 - val_accuracy: 0.2100 - val_loss: 1.7871


In [21]:
model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2116 - loss: 1.8135


[1.7998183965682983, 0.21060000360012054]